# Decorators

An overview on the decorator implementation in Python oriented to a practical usage.

Outilne of basics:

- [Basic Implementation](#basic-implementation)
- [Passing Arguments](#passing-arguments)
- [Returning Values](#returning-values)
- [Introspection](#introspection)
- [Real Examples](#real-examples)

Fancy decorators outline:

- [Decorating Classes](#decorating-classes)
- [Nesting Decorators](#nesting-decorators)
- [Decorators with Arguments](#decorators-with-arguments)
- [Decorators w/wo Arguments](#decorators-wwo-arguments)
- [Stateful Decorators](#stateful-decorators)
- [Classes as Decorators](#classes-as-decorators)
- [More Real Examples](#more-real-examples)

Exercises:

1. Create a decorator that caches the results of a function call.
2. Write a decorator that logs the execution time of a function and prints a warning if it exceeds a given threshold.
3. Implement a decorator that checks if a user is in a list of allowed users before executing a function.

## Basic Implementation

`#1` and `#2` are equivalent implementations, `#2` takes advantage of pythonic syntax sugar:

```python
def decorator(function):
	def wrapper():
		# operations before running function
		...
		function()
		# operations after run function
		...
	return wrapper

# Implementation #1
def function():
	...
function = decorator(function)
function()

# Implementation #2 (syntax sugar)
@decorator
def function():
	...
function()
```

In [16]:
def decorator(func):
    def wrapper():
        print(">> Before function")
        func()
        print(">> After function")
    return wrapper

@decorator
def func():
    print("Running function")

func()

>> Before function
Running function
>> After function


## Passing Arguments

Use [`*args` and `**kwargs`](https://realpython.com/python-kwargs-and-args/) in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments.

In [1]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [2]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

print(return_greeting("Adam"))

Creating greeting
Creating greeting
None


## Returning Values

In [1]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [2]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

print(return_greeting("Adam"))

Creating greeting
Creating greeting
Hi Adam


## Introspection

Introspection is the ability of an object to know about its own attributes at runtime. 

Decorators should use the [`@functools.wraps`](https://docs.python.org/library/functools.html#functools.wraps) decorator, which will preserve information about the original function.

In [19]:
import functools

def do_twice(func):
    @functools.wraps(func) # it preserves information about the original function
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def function():
    print("running function")

In [20]:
function

<function __main__.function()>

In [21]:
function.__name__

'function'

In [22]:
help(function)

Help on function function in module __main__:

function()



## Real Examples

### Timing Decorator

In [23]:
from time import time

def timing(func):
    def wrapper(*args, **kwargs):  #provide value to decorator
        t0 = time()
        a = func(*args, **kwargs)
        t1 = time()
        print('{} Time elapsed: {:.3f} s'.format(func.__name__, t1-t0))
        return a # return value from decorator
    return wrapper
    
@timing
def SumEverything(*args):
    return sum([_ for _ in args]) 

arr = [i for i in range(10000)]
print('sum equal to', SumEverything(*arr))

SumEverything Time elapsed: 0.000 s
sum equal to 49995000


### Debugging Decorator

See number into comments below:

1. Create a list of the positional arguments. Use `repr()` to get a nice string representing each argument.
2. Create a list of the keyword arguments. The [f-string](https://realpython.com/python-f-strings/) formats each argument as `key=value` where the `!r` specifier means that `repr()` is used to represent the value.
3. The lists of positional and keyword arguments is joined together to one signature string with each argument separated by a comma.
4. The return value is printed after the function is executed.

In [24]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

### Slow Down Decorator

In [25]:
import functools
import time

def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)
        
countdown(3)

3
2
1
Liftoff!


# Fancy Decorators

## Decorating Classes

Two ways to use decorators on classes:

1. decorate the methods of the class
2. decorate the whole class

In [ ]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535
    
# Also uding custom decorating functions

class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [ ]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str

## Nesting Decorators

In [ ]:
@debug
@do_twice
def greet(name):
    print(f"Hello {name}")

## Decorators with Arguments

In [ ]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

## Decorators w/wo Arguments

When a decorator uses arguments, you need to add an extra outer function. The challenge is for your code to figure out if the decorator has been called with or without arguments. Since the function to decorate is only passed in directly if the decorator is called without arguments, the function must be an optional argument. This means that the decorator arguments must all be specified by keyword. You can enforce this with the special * syntax, which means that all following parameters are keyword-only:

1. If `name` has been called without arguments, the decorated function will be passed in as `_func`. If it has been called with arguments, then `_func` will be `None`, and some of the keyword arguments may have been changed from their default values. The `*` in the argument list means that the remaining arguments can’t be called as positional arguments.
2. In this case, the decorator was called with arguments. Return a decorator function that can read and return a function.
3. In this case, the decorator was called without arguments. Apply the decorator to the function immediately.

```python
def name(_func=None, *, kw1=val1, kw2=val2, ...):  # 1
    def decorator_name(func):
        ...  # Create and return a wrapper function.

    if _func is None:
        return decorator_name                      # 2
    else:
        return decorator_name(_func)               # 3
```

In [ ]:
def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)
    
@repeat
def say_whee():
    print("Whee!")

@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")    

## Stateful Decorators

A decorator that **can keep track of state**. 

In [28]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1  # state of the function
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_whee():
    print("Whee!")

In [34]:
say_whee()

Call 5 of 'say_whee'
Whee!


## Classes as Decorators

The typical way to <u>maintain state</u> is by using classes.

In [4]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_whee():
    print("Whee!")

## More Real Examples

### Slow-Down Upgraded

In [35]:
import functools
import time

# ...

def slow_down(_func=None, *, rate=1):
    """Sleep given amount of seconds before calling the function"""
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)
        return wrapper_slow_down

    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)

@slow_down(rate=2)
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

### Creating Singletons

In [36]:
import functools

# ...

def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if wrapper_singleton.instance is None:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton

@singleton
class TheOne:
    pass

first = TheOne()
second = TheOne()

print(first is second)

True


### Caching Return Values

In [5]:
import functools

# ...

def cache(func):
    """Keep a cache of previous function calls"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = {}
    return wrapper_cache

@cache
@CountCalls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

fibonacci(5)

Call 1 of 'fibonacci'
Call 2 of 'fibonacci'
Call 3 of 'fibonacci'
Call 4 of 'fibonacci'
Call 5 of 'fibonacci'
Call 6 of 'fibonacci'


5

In [6]:
# Cached values
fibonacci.cache

{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5}

### Adding Information About Users

In [ ]:
# ...

def set_unit(unit):
    """Register a unit on a function"""
    def decorator_set_unit(func):
        func.unit = unit
        return func
    return decorator_set_unit

In [ ]:
import functools
import pint

# ...

def use_unit(unit):
    """Have a function return a Quantity with given unit"""
    use_unit.ureg = pint.UnitRegistry()
    def decorator_use_unit(func):
        @functools.wraps(func)
        def wrapper_use_unit(*args, **kwargs):
            value = func(*args, **kwargs)
            return value * use_unit.ureg(unit)
        return wrapper_use_unit
    return decorator_use_unit

### Validating `json`

In [ ]:
import functools
from flask import abort
from requests import request

def validate_json(*expected_args):
    def decorator_validate_json(func):
        @functools.wraps(func)
        def wrapper_validate_json(*args, **kwargs):
            json_object = request.get_json()
            for expected_arg in expected_args:
                if expected_arg not in json_object:
                    abort(400)
            return func(*args, **kwargs)
        return wrapper_validate_json
    return decorator_validate_json

In [ ]:
import functools
from flask import Flask, request, abort

app = Flask(__name__)

# ...

@app.route("/grade", methods=["POST"])
@validate_json("student_id")
def update_grade():
    json_data = request.get_json()
    # Update database.
    return "success!"